In [1]:
import pandas as pd

import os

import base64
import requests
import json

from urllib.parse import urlencode
from lyricsgenius import Genius

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import fasttext as ft
import translators as ts
import translators.server as tss

import textwrap

from flair.models import TextClassifier
from flair.data import Sentence

import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.probability import FreqDist

import string

from matplotlib import pyplot as plt
from wordcloud import WordCloud

from pathlib import Path

from PIL import Image
from random import randint

Using state Castille and LeÃ³n server backend.
/home/roque/01. IronHack/00. Data Analytics/01. Course/36. Week 12 - Day 3/git/env-mid-bootcamp-project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv(Path('data/dataset.csv'))
dataset

,Artist,Gender,Genre,Band,Lyrics,Language,Lyrics EN,Sentiment,top words
0,Taylor Swift,female,pop,no,NaN,NaN,NaN,NaN,NaN
1,Bad Bunny,male,reggaeton,no,NaN,NaN,NaN,NaN,NaN
2,Drake,male,hip hop,no,NaN,NaN,NaN,NaN,NaN
3,The Weeknd,male,rnb,no,NaN,NaN,NaN,NaN,NaN
4,BTS,male,K-pop,yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,Oliver Tree,male,electronic,no,NaN,NaN,NaN,NaN,NaN
96,Polo G,male,rap,no,NaN,NaN,NaN,NaN,NaN
97,Zach Bryan,male,country,no,NaN,NaN,NaN,NaN,NaN
98,Jhayco,male,rap,no,NaN,NaN,NaN,NaN,NaN


In [3]:
dataset[['Lyrics','Language','Lyrics EN','Sentiment','top words']] = dataset[['Lyrics','Language','Lyrics EN','Sentiment','top words']].astype(object)

In [4]:
dataset.dtypes

Artist       object
Gender       object
Genre        object
Band         object
Lyrics       object
Language     object
Lyrics EN    object
Sentiment    object
top words    object
dtype: object

In [5]:
'''SECRETS'''
def read_secrets():
    keys={}
    #keys_file = open('secrets.txt','r') #Had errors using this within a virtual environment
    keys_file = os.fdopen(os.open('secrets.txt', os.O_RDONLY))
    for line in keys_file:
        (key, val) = line.replace('\n','').split("|")
        keys[key] = val
    return keys

In [6]:
#SPOTIFY

In [7]:
'''FUNCTION TO GET THE TOKEN FOR SPOTIFY API'''

def get_token_spotify(key):
    client_creds = f"{key['spotifyId']}:{key['spotifySecret']}"
    client_creds_b64 = base64.b64encode(client_creds.encode())
    
    headers_token = {"Authorization":f"Basic {client_creds_b64.decode()}"}
    data_token = {"grant_type":"client_credentials"}
 
    return requests.post('https://accounts.spotify.com/api/token', data=data_token, headers=headers_token).json()['access_token']

In [8]:
'''FUNCTION TO GET THE ID OF ONE ARTIST'''

def get_artist_id_spotify(name):
    response_token = get_token_spotify(read_secrets())

    headers = {"Authorization":f"Bearer {response_token}"}
    endpoint = 'https://api.spotify.com/v1/search'
    data = urlencode({
        "q": name,
        "type": "artist"
    })

    lookup_url = f"{endpoint}?{data}"

    request = requests.get(lookup_url, headers=headers)

    return request.json()['artists']['items'][0]['id']

In [9]:
'''FUNCTION TO GET THE TOP (5) SONGS OF ONE ARTIST'''

def get_top_songs_artist_spotify(artistId, n_songs=5):
    n_songs=(10 if (n_songs>10) else n_songs)
    
    response_token = get_token_spotify(read_secrets())
    headers = {'Authorization': f'Bearer {response_token}',
               'Content-Type': 'application/json'}

    data = urlencode({
        "market": "US"
    })
    endpoint = f"https://api.spotify.com/v1/artists/{get_artist_id_spotify(artistId)}/top-tracks"
    lookup_url = f"{endpoint}?{data}"

    request = requests.get(lookup_url, headers=headers)
    
    return [request.json()['tracks'][i]['name'] for i in range(n_songs) ]

In [10]:
#GENIUS

In [11]:
'''
FUNCTION TO GET THE ACCESS TOKEN FOR GENIUS

As I had a lot of problems to get the Access Token directly from the Genius API and from the library 
lyricsgenius, I decided to use selenium to get the access token and then continue with lyricsgenius
'''

def get_token_genius(key):
    
    # Url
    
    params = urlencode({
        "client_id": key['geniusId'],
        "redirect_uri": key['geniusRedirectUri'],
        "scope": "me",
        "state": "2",
        "response_type": "token"
    })

    endpoint = f"https://api.genius.com/oauth/authorize"
    lookup_url = f"{endpoint}?{params}"

    # open the website
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))    
    driver.get(lookup_url)
    
    
    #Accept cookies

    cookies = driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')
    cookies.click()

    time.sleep(1)

    #Login

    email='roque.ourense@gmail.com'
    password='Roque76bis_'

    signin = driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/a[2]')
    signin.click()

    time.sleep(1)

    email_box = driver.find_element(By.ID, "user_session_login") 

    email_box.clear()

    email_box.send_keys(email)

    time.sleep(1)

    pass_box = driver.find_element(By.ID, 'user_session_password') 

    pass_box.clear()

    pass_box.send_keys(password)

    time.sleep(1)

    login = driver.find_element(By.ID,'user_session_submit')
    login.click()

    time.sleep(1.5)

    approve = driver.find_element(By.XPATH,'/html/body/div[4]/div[3]/form/input[6]')
    approve.click()

    url_callback = driver.current_url

    return url_callback[url_callback.find('access_token=') + len('access_token='):url_callback.find('&state=')]


In [12]:
#Lyrics

In [13]:
'''
FUNCTION TO TO DOWNLOAD UP TO 5 LYRICS (OF 10 SONGS) FROM GENIUS
'''

def get_lyrics_from_genius(artist):

    genius = Genius(read_secrets()['geniusAccessToken'])

    # Remove section headers (e.g. [Chorus]) from lyrics when searching
    genius.remove_section_headers = True
    genius.skip_non_songs = False

    s=1        
    lyrics_list = []

    for songs in get_top_songs_artist_spotify(artist,10):                   
        try:            
            print('=== '+str(s)+' ===')

            song = genius.search_song(artist=artist, title=songs.split(' - ')[0])
            lyric = song.lyrics.split(songs.split(' - ')[0] + ' Lyrics')[1].replace('\n',' ').split('Embed')[0].rstrip('0123456789').strip()

            lyrics_list.append(lyric)

            s+=1
        except:
            pass

        if (s > 5):
            break

    return lyrics_list
    

In [14]:
''''
FUNCTION TO GET THE LANGUAGE(S) OF THE LYRICS    
'''

def fasttext_detect_language(text):
    
    # Load the pretrained model

    model = ft.load_model("data/fasttext_pretrained_model/lid.176.bin")

    text = text.replace('\n', " ")
    prediction = model.predict([text])

    return prediction[0][0][0][-2:]

In [15]:
'''
FUNCTION TO TRANSLATE LYRICS OF ANY LANGUAGE TO ENGLISH
'''

def translate_lyrics(lyric):
    
    ts.translators_pool

    lyrics_en = ''
    
    from_language = fasttext_detect_language(lyric)
        
    if (from_language == 'en'):
        lyrics_en = lyric
    else:   
        if (len(lyric)>2000):
            lyric_splited_en = []
            lyric_splited = textwrap.wrap(lyric, 2000, break_long_words=False)
            for line in lyric_splited:
                lyric_splited_en.append(tss.google(line, from_language, 'en'))
            lyrics_en = ' '.join(lyric_splited_en)
        else:
            try:
                lyrics_en = tss.google(lyric, from_language, 'en')
            except:
                try:
                    lyrics_en = ts.translate_text(lyric, from_language, 'en')
                except:
                    lyrics_en = lyric
    return lyrics_en

In [16]:
'''
FUNCTION TO CALCULATE THE SENTIMENT OF ONE TEXT
'''

def calculate_sentiment(text):
    classifier = TextClassifier.load('en-sentiment')

    sentence = Sentence(text)
    
    classifier.predict(sentence)

    if (sentence.labels[0].value=='NEGATIVE'):
        sentiment = -sentence.labels[0].score
    else:
        sentiment = sentence.labels[0].score
    
    return sentiment

In [17]:
'''
FUNCTION TO CALCULATE THE MOST COMMON WORDS IN ONE TEXT
'''

def get_top_words(text):
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('omw-1.4')

    lyrics_clean = []
        
    wordnet_lemmatizer = WordNetLemmatizer()

    stop_words = list(set(stopwords.words('english')))
    stop_words.extend(['yeah','uh','myyyyyyyet','oh','ooh', 'im', 'dont', 'na', 'one', 'wan', 'cant', 'cause', 'youre', 'aint', 'gon', 'might', 'wont', 'ah', 'still', 'ive', 'em', 'hey', '’', 'without', 'thats', 'also', 'every', 'eh', 'nothing', 'ayy', 'id', 'another', 'ever', 'okay', 'til', 'oohooh', 'pa', 'u', 'everybody', 'ima', 'someone', 'around', 'ey', 'woah', 'maybe', 'two', 'already', 'ohoh', 'everything', 'nobody', 'really', 'bum', 'woo', 'ya', 'eheh', 'much', 'alright', 'somebody', 'huh', 'something', 'seem', 'as', 'didnt', 'there', 'tryna', 'else', 'mmm', '21', 'la', 'shes', 'ho', 'youve', 'ohohoh', 'youll', 'oohoohooh', 'nothin', 'buh', 'somethin', 'sometimes', 'ere', 'many', 'para', 'anyone', 'whats', 'rather', 'everyone', 'hahahahaha', 'whatever', 'since', 'whole', 'ahah', 'lil', 'yakayaka', 'wow', 'though', 'three', 'anything', 'yeahyeah', 'twenty', 'next', 'le', 'prr', 'hi', 'control', 'probably', 'sugar', 'bottle', 'straight', 'kun', 'pas', 'c', 'couldnt', 'yo', 'haha', 'apart', 'he', '40', 'tra', 'ahahah', 'tomorrow', 'doesnt', 'juda', 'ddu', 'along', 'hundred', 'yall', 'youd', 'tá', 'eighteen', 'yet', 'vamo', 'outta', 'wasnt', 'ready', 'yah', 'isnt',  'gaga', 'brr', 'uah', 'mon', 'nah', 'million', 'may', 'de', 'ohohohoh', 'weve', 'pon', 'although', 'anymore', 'finally', 'yeh', 'uhhuh', 'hoo', 'someday', 'pal', 'um', 'ba', 'double', 'kinda', 'ago', 'somewhere', 'somo', 'wantin', 'teri', 'ten', 'hmm', 'five', 'vo—', 'behind', 'wuh', 'more—', 'wohoh', 'shouldnt', 'your—', 'thousand', 'hai', 'middle', 'theyre', 'ohohohohoh', 'ahahahah', '‘', 'ti', 'lala', 'pie', 'wouldnt', 'whenever', 'wave',  'everybodys', 'forty', 'guayar', 'tera', 'uhuh', 'brrt', 'g5', 'judaahah', 'h', 'doodoodoodoo', 'da', 'nanananananana', 'aw', 'twice', 'fifty', 'tu', 'dadadada', 'dundada', 'dadundada', 'lalalala', 'lalala', 'iii', 'suppose', 'shouldve', 'dada', 'ha', 'whered', 'opps', 'dynnanana', 'rarauw', 'hop', 'ki', 'rot', 'yuh', 'nine', 'thick', 'dumb', 'rudeboyz', 'tití', 'my—', 'mm',  'que', 'apa', 'vemo', 'whoa', 'tha', 'ak', 'dadada', 'thirtyfive', 'nono',   'kesariya', 'ishq', 'piya', 'jo', 'khuda', 'ohooh', 'oooh', 'theyll', 'everywhere', 'hah', 'woh', 'actin', 'lalalalalalala', 'byza', 'whoawhoawhoawhoawhoa', 'boo', '4', 'vroom', 'thatthatthat', 'instead', '\u200bnobody', 'tarawill', 'downdown', 'somehow', 'half', 'alight', 'hahahaha', 'whyd', 'heh', 'baow', 'yandel', 'patrá', 'solo', 'here', 'doodoodoo', 'mi', 'ee', 'tata', 'xanny', 'chee', 'x2', 'grrt', 'arent', 'woulda', '내', 'danananananana', 'upon', 'youim', 'cel', 'vece', 'rabta', 'como', 'who', 'acura', 'hunnid', 'khudgarzi', 'wohi', 'quiero', 'una', 'en', 'eee', 'ohwoahwoah', 'engine', 'vemor', 'humacao', 'ferragamo', '네', 'brratatata', 'trratatata', 'ddudduddu',  '”', 'wetto', 'grey59', 'billion', 'hawaiiii', 'ratata', 'nahi', 'wahi', 'dumdumdum', 'dadadumdumdum', 'dadadumdum', 'onenight', 'toa', 'couldve', 'woahohoh', 'nanana', 'havent', 'quite', 'eheheh', 'plo', 'ohwoah', 'mix', 'jaun', 'hath', 'se', 'kuch', 'w', 'maybach',  'aaja', 'mwhen', 'yeaheh',  'ajá', 'chica', 'latinas', 'you—', 'usual', 'ahooh', 'simple', 'amall', 'ehoh', 'luis', 'r', '“', 'wohohohoh', 'iti', '100', 'í', 'hahaha', 'grr',  'whoo', 'itll', 'yaah', 'je', '6ix', 'nanananana', 'woohoo', '날아가', '나', '너를', '너는', '이',  'ohh', 'noti', 'ii', 'aaa', '6', 'eight', 'herehow', 'tre', 'bumbum', 'blamblam', 'manaun', 'ke', 'toh', 'tujh', 'ohohohohohoh', 'sususummertime',  'gwagen', 'kabira', 'jaa', 'faqeera', 'ja', 'kaisa', 'dekhe', 'goodbyeif', 'nio', 'test—', 'natti', 'oye', 'así', 'f', 'mustve', 'taba', 'dadida', 'hmmmm', 'rvssian', 'diddy', 'blp', 'pumpumpum', 'gaby',  'fifth', 'stepfifth', 'mе', 'mía', 'ultra', '간판', '내리고', '문', '잠가', 'dududun', 'dulululun', 'bada', 'denynynyny', 'sisiside', 'anytime', 'ol', 'down\u200b', 'agua', 'twelve', 'wohohoh', 'youi', 'tere', 'khaali', 'samaaya', 'x5', 'dadadum', 'ow', 'shoodoodoodoodoo', '\u200blrig', 'uoy', 'chimba', 'maluma', 'tt', 'mhm', 'lo', 'uhyeahyeahyeah', 'chingo', '1950s', 'chamaquita', 'thisll', 'a—', 'yeahyeahyeahyeah', 'ayo', 'pimp', 'inner', 'wherever',  'herei', 'bhi', 'jaisi', 'mujhko', 'youif'])
    stop_words.extend(['mmmm', 'from…', 'dadadadadadada', 'lou', 'cubano', 'solitai', 'ugh', 'm', 'mainly', '505', 'sevenhour', 'fortyfiveminute', 'andar', 'v8', 'j', 'youtell', 'youhe', 'zúmbale', 'lalalatinas', 'estoy', 'buscando', 'muévelo', 'shoulda', 'woowoowoo', 'mai', 'punto', 'andyou', 'pr', 'marimba', 'elsei', 'seventeen', 'void', 'anywhere', 'mil', 'uy', 'bellaco', 'pupupupum', 'gotti', 'aheh', 'mala', 'ía', 'rosa', 'andvemo', 'thathe', 'youthat', '자', 'thatll', '날', 'doodoo', 'thy', 'pro', 'maam', 'pic', 'thirty', 'wholl', 'happenedwohohoh', 'meand', 'riide', 'hulahula', 'hula', 'jewelerjeweler', 'skyyy', 'stopop', 'duko', 'lao', 'coroná', 'uhoh', 'able', 'mmmjum', 'cap', 'ra', 'bodyody', 'r'])
        
    new_text = text.translate(str.maketrans('', '', string.punctuation)).lower()
        
    word_tokens = nltk.tokenize.word_tokenize(new_text) 

    for word in word_tokens: 
        if word not in stop_words: 
            lyrics_clean.append(word) 

    lemma_words = []

    for word in lyrics_clean:
        word1 = wordnet_lemmatizer.lemmatize(word, pos = "n") #nouns
        word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v") #verbs
        word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a")) #adjectives
        lemma_words.append(word3)

    #find the frequency of words
    fdist = FreqDist(lemma_words)

    return fdist.most_common(50)

In [18]:
'''
FUNCTION TO GENERATE A WORD CLOUD WITH A TEXT

'''

def get_word_cloud(text_list):
    #Convert word list to a single string
    clean_words_string = " ".join([tup[0] for tup in text_list])

    #generating the wordcloud with a shape

    shape_list=os.listdir(Path('data/shapes/'))
                
    image_route=f"data/shapes/{shape_list[randint(0,len(shape_list))-1]}"              
    mask = np.array(Image.open(Path(image_route)))

    wordcloud = WordCloud(max_font_size=50, max_words=50, background_color="white", mask=mask, contour_color='#023075',contour_width=3).generate(clean_words_string)

    #plot the wordcloud
    plt.figure(figsize = (6, 6))
    plt.imshow(wordcloud)

    #to remove the axis value
    plt.axis("off")
    plt.show()

In [19]:
#Process

In [21]:
'''
Try to find the lyrics of every artist
'''

lyrics_dict = dict()   
    
for art in dataset['Artist']:
    lyrics_dict[art] = []
    lyrics_dict[art].append(get_lyrics_from_genius(art))

=== 1 ===
Searching for "Anti-Hero" by Taylor Swift...
Done.
=== 2 ===
Searching for "Lavender Haze" by Taylor Swift...
Done.
=== 3 ===
Searching for "Midnight Rain" by Taylor Swift...
Done.
=== 4 ===
Searching for "Snow On The Beach (feat. Lana Del Rey)" by Taylor Swift...
Done.
=== 4 ===
Searching for "Maroon" by Taylor Swift...
Done.
=== 5 ===
Searching for "You're On Your Own, Kid" by Taylor Swift...
Done.
=== 5 ===
Searching for "Bejeweled" by Taylor Swift...
Done.
=== 1 ===
Searching for "Me Porto Bonito" by Bad Bunny...
Done.
=== 2 ===
Searching for "Tití Me Preguntó" by Bad Bunny...
Done.
=== 3 ===
Searching for "Efecto" by Bad Bunny...
Done.
=== 4 ===
Searching for "Ojitos Lindos" by Bad Bunny...
Done.
=== 5 ===
Searching for "Neverita" by Bad Bunny...
Done.
=== 1 ===
Searching for "Rich Flex" by Drake...
Done.
=== 2 ===
Searching for "Jimmy Cooks (feat. 21 Savage)" by Drake...
Done.
=== 2 ===
Searching for "Pussy & Millions (feat. Travis Scott)" by Drake...
Done.
=== 2 ===
Se

Done.
=== 4 ===
Searching for "Bored" by Billie Eilish...
Done.
=== 5 ===
Searching for "when the party's over" by Billie Eilish...
=== 5 ===
Searching for "bad guy" by Billie Eilish...
Done.
=== 1 ===
Searching for "I'm Good (Blue)" by David Guetta...
Done.
=== 1 ===
Searching for "Crazy What Love Can Do" by David Guetta...
Done.
=== 2 ===
Searching for "DON'T YOU WORRY" by David Guetta...
Done.
=== 2 ===
Searching for "Love Tonight" by David Guetta...
Done.
=== 2 ===
Searching for "Titanium (feat. Sia)" by David Guetta...
Done.
=== 2 ===
Searching for "Remember (and David Guetta)" by David Guetta...
Done.
=== 2 ===
Searching for "Heartbreak Anthem (with David Guetta & Little Mix)" by David Guetta...
Done.
=== 2 ===
Searching for "Memories (feat. Kid Cudi)" by David Guetta...
Done.
=== 2 ===
Searching for "Sexy Bitch (feat. Akon)" by David Guetta...
Done.
=== 2 ===
Searching for "BED" by David Guetta...
Done.
=== 1 ===
Searching for "Yellow" by Coldplay...
=== 1 ===
Searching for "Spa

Done.
=== 4 ===
Searching for "Tera Hone Laga Hoon" by Pritam...
No results found for: 'Tera Hone Laga Hoon Pritam'
=== 4 ===
Searching for "Kalank (Title Track)" by Pritam...
No results found for: 'Kalank (Title Track) Pritam'
=== 4 ===
Searching for "Rasiya" by Pritam...
Done.
=== 1 ===
Searching for "LA CANCIÓN" by J Balvin...
Done.
=== 2 ===
Searching for "UN PESO" by J Balvin...
Done.
=== 3 ===
Searching for "No Me Conoce" by J Balvin...
Done.
=== 3 ===
Searching for "QUE PRETENDES" by J Balvin...
Done.
=== 3 ===
Searching for "I Like It" by J Balvin...
Done.
=== 4 ===
Searching for "AM Remix" by J Balvin...
Done.
=== 5 ===
Searching for "Una Locura" by J Balvin...
Done.
=== 1 ===
Searching for "La Santa" by Daddy Yankee...
Done.
=== 2 ===
Searching for "X ÚLTIMA VEZ" by Daddy Yankee...
Done.
=== 3 ===
Searching for "Gasolina" by Daddy Yankee...
Done.
=== 4 ===
Searching for "Lo Que Pasó, Pasó" by Daddy Yankee...
Done.
=== 5 ===
Searching for "Despacito" by Daddy Yankee...
=== 5 =

Done.
=== 1 ===
Searching for "Quevedo: Bzrp Music Sessions, Vol. 52" by Bizarrap...
Done.
=== 2 ===
Searching for "Duki: Bzrp Music Sessions, Vol. 50" by Bizarrap...
Done.
=== 3 ===
Searching for "Tiago PZK: Bzrp Music Sessions, Vol. 48" by Bizarrap...
Done.
=== 4 ===
Searching for "Villano Antillano: Bzrp Music Sessions, Vol. 51" by Bizarrap...
Done.
=== 5 ===
Searching for "Eladio Carrion: Bzrp Music Sessions, Vol. 40" by Bizarrap...
Done.
=== 5 ===
Searching for "Sin Frenos" by Bizarrap...
Done.
=== 1 ===
Searching for "LA INOCENTE" by Mora...
Done.
=== 2 ===
Searching for "MEMORIAS" by Mora...
Done.
=== 3 ===
Searching for "APA" by Mora...
Done.
=== 4 ===
Searching for "Volando" by Mora...
Done.
=== 5 ===
Searching for "Una Vez" by Mora...
Done.
=== 5 ===
Searching for "COMO HAS ESTAU?" by Mora...
Done.
=== 5 ===
Searching for "LACONE" by Mora...
Done.
=== 1 ===
Searching for "Shut Down" by BLACK PINK...
Done.
=== 2 ===
Searching for "Pink Venom" by BLACK PINK...
Done.
=== 3 ===
S

=== 3 ===
Searching for "Eastside (with Halsey & Khalid)" by Khalid...
Done.
=== 3 ===
Searching for "Better" by Khalid...
Done.
=== 4 ===
Searching for "Location" by Khalid...
Done.
=== 5 ===
Searching for "8TEEN" by Khalid...
Done.
=== 1 ===
Searching for "La Bachata" by Manuel Turizo...
Done.
=== 2 ===
Searching for "Los Cachos" by Manuel Turizo...
Done.
=== 3 ===
Searching for "Éxtasis" by Manuel Turizo...
Done.
=== 4 ===
Searching for "Una Vaina Loca" by Manuel Turizo...
Done.
=== 5 ===
Searching for "Desconocidos" by Manuel Turizo...
Done.
=== 1 ===
Searching for "Just Wanna Rock" by Lil Uzi Vert...
Done.
=== 2 ===
Searching for "20 Min" by Lil Uzi Vert...
Done.
=== 3 ===
Searching for "XO Tour Llif3" by Lil Uzi Vert...
=== 3 ===
Searching for "The Way Life Goes (feat. Oh Wonder)" by Lil Uzi Vert...
Done.
=== 3 ===
Searching for "Money Longer" by Lil Uzi Vert...
Done.
=== 4 ===
Searching for "Erase Your Social" by Lil Uzi Vert...
Done.
=== 5 ===
Searching for "Neon Guts (feat. Ph

Done.
=== 3 ===
Searching for "Super Bass" by Nicki Minaj...
Done.
=== 4 ===
Searching for "Tusa" by Nicki Minaj...
Done.
=== 5 ===
Searching for "Starships" by Nicki Minaj...
Done.
=== 1 ===
Searching for "Siempre Pendientes" by Luis R Conriquez...
Done.
=== 2 ===
Searching for "JGL" by Luis R Conriquez...
Done.
=== 2 ===
Searching for "Los Botones Azules" by Luis R Conriquez...
Done.
=== 3 ===
Searching for "El Gavilán" by Luis R Conriquez...
No results found for: 'El Gavilán Luis R Conriquez'
=== 3 ===
Searching for "Puro Campeón" by Luis R Conriquez...
Done.
=== 4 ===
Searching for "Solita (En Vivo)" by Luis R Conriquez...
Done.
=== 4 ===
Searching for "Me Metí En El Ruedo" by Luis R Conriquez...
Done.
=== 4 ===
Searching for "El Buho" by Luis R Conriquez...
Done.
=== 5 ===
Searching for "Los Mire Con Talento" by Luis R Conriquez...
Done.
=== 5 ===
Searching for "Y Si Me Miran" by Luis R Conriquez...
Done.
=== 1 ===
Searching for "One Kiss (with Dua Lipa)" by Calvin Harris...
Done.

Done.
=== 2 ===
Searching for "For Whom The Bell Tolls (Remastered)" by Metallica...
Done.
=== 2 ===
Searching for "One (Remastered)" by Metallica...
Done.
=== 2 ===
Searching for "Sad But True (Remastered)" by Metallica...
Done.
=== 2 ===
Searching for "Fuel" by Metallica...
Done.
=== 1 ===
Searching for "Me Porto Bonito" by Chencho Corleone...
Done.
=== 2 ===
Searching for "La Llevo Al Cielo (Ft. Ñengo Flow)" by Chencho Corleone...
Done.
=== 2 ===
Searching for "Desesperados" by Chencho Corleone...
Done.
=== 3 ===
Searching for "El Efecto" by Chencho Corleone...
Done.
=== 4 ===
Searching for "Una Locura" by Chencho Corleone...
Done.
=== 5 ===
Searching for "Nos Comemos Vivos" by Chencho Corleone...
Done.
=== 1 ===
Searching for "Closer" by Halsey...
=== 1 ===
Searching for "Without Me" by Halsey...
=== 1 ===
Searching for "Boy With Luv (feat. Halsey)" by Halsey...
Done.
=== 1 ===
Searching for "Eastside (with Halsey & Khalid)" by Halsey...
Done.
=== 1 ===
Searching for "Him & I (with

In [22]:
'''
language and translations
'''

for artist in dataset['Artist']:
    lang_set = set()
    lyric_en = []
    
    if len(lyrics_dict[artist])==1 or artist!='Motat':
        for lyric in lyrics_dict[artist][0]:
            print(artist)
            lang_set.add(fasttext_detect_language(lyric))
            lyric_en.append(translate_lyrics(str(lyric)))

    lyrics_dict[artist].append(lang_set)
    lyrics_dict[artist].append(lyric_en)

Taylor Swift
Taylor Swift
Taylor Swift


Taylor Swift
Taylor Swift


Bad Bunny
Bad Bunny


Bad Bunny


Bad Bunny


...

Jhayco


Jhayco


Pop Smoke
Pop Smoke


Pop Smoke
Pop Smoke


Pop Smoke


In [23]:
'''
Calculate the sentiment of every lyric
'''

for artist in dataset['Artist']:
    sentiment_list = []
    
    if len(lyrics_dict[artist])>2:
        for lyric in lyrics_dict[artist][2]:
            sentiment_list.append(calculate_sentiment(str(lyric)))        
            
    lyrics_dict[artist].append(sentiment_list)

2022-12-15 15:05:29,009 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:33,394 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:36,852 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:40,207 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:43,807 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:47,513 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:51,040 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:54,761 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:05:58,306 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:06:01,866 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:06:05,114 loadin

2022-12-15 15:11:49,296 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:11:53,978 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:11:58,716 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:03,015 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:08,369 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:14,284 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:20,342 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:26,893 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:34,445 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:40,973 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:12:47,498 loadin

2022-12-15 15:18:59,097 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:03,124 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:07,179 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:10,780 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:14,459 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:19,009 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:23,113 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:28,331 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:34,229 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:39,719 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:19:44,016 loadin

2022-12-15 15:25:25,072 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:29,074 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:33,132 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:37,513 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:41,763 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:45,767 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:49,652 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:53,465 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:25:57,974 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:26:02,898 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:26:08,172 loadin

2022-12-15 15:31:57,738 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:02,713 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:07,283 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:11,316 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:15,012 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:19,189 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:23,367 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:27,660 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:32,491 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:37,146 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:32:41,638 loadin

2022-12-15 15:38:42,122 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:38:46,559 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:38:51,566 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:38:57,050 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:02,171 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:06,890 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:11,267 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:15,391 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:20,174 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:25,685 loading file /home/roque/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-12-15 15:39:31,190 loadin

In [24]:
'''
Calculate the top words of every lyric
'''

for artist in dataset['Artist']:
    all_lyrics = []
    
    if len(lyrics_dict[artist])>2:
        for lyric in lyrics_dict[artist][2]:      
            all_lyrics.append(get_top_words(lyric))
        
    lyrics_dict[artist].append(all_lyrics)

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

...

[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/roque/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/roque/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

In [25]:
'''
Export to csv to use during analysis
'''

def return_lyrics_dictionary(artist):
    return lyrics_dict[artist][0]

def return_language_dictionary(artist):
    return lyrics_dict[artist][1]

def return_lyrics_en_dictionary(artist):
    return lyrics_dict[artist][2]

def return_sentiment_dictionary(artist):
    return lyrics_dict[artist][3]

def return_top_words_dictionary(artist):
    return lyrics_dict[artist][4]

dataset_full = dataset
dataset_full['Lyrics'] = dataset_full.apply(lambda x: return_lyrics_dictionary(x['Artist']), axis=1)
dataset_full['Language'] = dataset_full.apply(lambda x: return_language_dictionary(x['Artist']), axis=1)
dataset_full['Lyrics EN'] = dataset_full.apply(lambda x: return_lyrics_en_dictionary(x['Artist']), axis=1)
dataset_full['Sentiment'] = dataset_full.apply(lambda x: return_sentiment_dictionary(x['Artist']), axis=1)
dataset_full['top words'] = dataset_full.apply(lambda x: return_top_words_dictionary(x['Artist']), axis=1)

dataset_full.to_csv(Path('data/dataset_full.csv'))

In [ ]:
get_word_cloud(lyrics_dict['Taylor Swift'][4][0])